# Fantasy Baseball Redraft

In [ ]:
from gym import spaces
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam
import pandas as pd
import gym

class SimpleFantasyBaseballEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self, player_stats, scoring_rules, positions, position_weights, num_teams=10):
        super(SimpleFantasyBaseballEnv, self).__init__()
        self.player_stats = player_stats
        self.scoring_rules = scoring_rules
        self.positions = positions
        self.position_weights = position_weights
        self.num_teams = num_teams
        self.num_rounds = sum(positions.values())
        
        self.action_space = spaces.Discrete(len(self.player_stats))
        self.observation_space = spaces.Discrete(self.num_teams * self.num_rounds)
        self.reset()
    
    def reset(self):
        self.teams = {i: {pos: [] for pos in self.positions} for i in range(self.num_teams)}
        self.available_players = list(self.player_stats.keys())
        
        self.available_players.sort(key=lambda player: (
            -self.player_stats[player].get('woba', 0), 
            -self.player_stats[player].get('p_win', 0)
        ))
        
        self.draft_order = self.generate_draft_order()
        self.current_pick = 0
        return np.zeros(len(self.player_stats[self.available_players[0]]) - 1)
    
    def generate_draft_order(self):
        draft_order = []
        for round in range(self.num_rounds):
            if round % 2 == 0:
                draft_order.extend(range(self.num_teams))
            else:
                draft_order.extend(range(self.num_teams - 1, -1, -1))
        return draft_order
    
    def step(self, action):
        if action < 0 or action >= len(self.available_players):
            raise ValueError(f"Invalid action: {action}. Must be within range [0, {len(self.available_players)-1}]")
        
        player = self.available_players[action]
        team_id = self.draft_order[self.current_pick]
        
        player_position = self.player_stats[player]['position']
        
        if player_position not in self.positions or len(self.teams[team_id][player_position]) >= self.positions[player_position]:
            return np.zeros(len(self.player_stats[player]) - 1), -1, self.current_pick >= len(self.draft_order) - 1, {}
        
        self.teams[team_id][player_position].append(player)
        self.available_players.remove(player)
        
        self.current_pick += 1
        reward = self._calculate_reward(team_id, player_position)
        done = self.current_pick >= len(self.draft_order)
        return np.array(list(self.player_stats[player].values())[1:]), reward, done, {}
    
    def _calculate_reward(self, team_id, player_position):
        team_reward = 0
        for pos in self.teams[team_id]:
            for player in self.teams[team_id][pos]:
                player_stats = self.player_stats[player]
                position_weight = self.position_weights.get(pos, 1)
                for stat, value in player_stats.items():
                    if stat == 'woba':
                        team_reward += self.scoring_rules[stat] * value * position_weight * 2
                    elif stat == 'p_win':
                        team_reward += self.scoring_rules[stat] * value * position_weight * 1.5
                    elif stat in self.scoring_rules:
                        team_reward += self.scoring_rules[stat] * value * position_weight
        return team_reward

    def render(self, mode='human'):
        print(f'Teams: {self.teams}')
        print(f'Current Pick: {self.current_pick}')

# Load player stats from Excel file
df = pd.read_excel('player_stats2.xlsx')

player_stats = df.set_index('name').T.to_dict()
for player, stats in player_stats.items():
    if 'position' in stats:
        stats['position'] = stats['position'].strip()

scoring_rules = {
    'home_run': 4,
    'r_total_stolen_base': 2,
    'woba': 100,
    'p_save': 5,
    'p_win': 6,
    'xobp': -3,
}

positions = {
    'C': 1,
    '1B': 2,
    '2B': 2,
    '3B': 2,
    'SS': 2,
    'OF': 5,
    'P': 10,
}

position_weights = {
    'C': 1,
    '1B': 2,
    '2B': 2,
    '3B': 2,
    'SS': 3,
    'OF': 4,
    'P': 5,
}

env = SimpleFantasyBaseballEnv(player_stats, scoring_rules, positions, position_weights)

model = Sequential()
model.add(Dense(8, input_dim=6, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer=Adam(learning_rate=0.01))

num_episodes = 75
gamma = 0.95
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.01

replay_buffer = []
buffer_size = 10000
batch_size = 32

for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, [1, 6])
    done = False
    pick_count = 0
    max_picks = env.num_rounds * env.num_teams

    while not done and pick_count < max_picks:
        if np.random.rand() <= epsilon:
            action = random.randrange(len(env.available_players))
        else:
            probabilities = model.predict(state).flatten()
            probabilities = np.nan_to_num(probabilities, nan=0.0)
            probabilities /= np.sum(probabilities)
            if np.isnan(probabilities).any() or np.any(probabilities < 0):
                action = random.randrange(len(env.available_players))
            else:
                action = np.random.choice(len(env.available_players), p=probabilities[:len(env.available_players)])
        
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, 6])
        
        replay_buffer.append((state, action, reward, next_state, done))
        if len(replay_buffer) > buffer_size:
            replay_buffer.pop(0)
        
        if len(replay_buffer) >= batch_size:
            minibatch = random.sample(replay_buffer, batch_size)
            for s, a, r, ns, d in minibatch:
                target = r
                if not d:
                    target = r + gamma * np.amax(model.predict(ns.reshape(1, 6)))
                
                target_f = model.predict(s.reshape(1, 6))
                target_f[0][a] = target
                model.fit(s.reshape(1, 6), target_f, epochs=1, verbose=0)
        
        state = next_state
        pick_count += 1
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Save the model
model.save("nn_fantasy_baseball_model")

# Load the model
model = tf.keras.models.load_model("nn_fantasy_baseball_model")


In [ ]:
# Simulate a draft
state = env.reset()
done = False
pick_count = 0  # Initialize pick counter
max_picks = env.num_rounds * env.num_teams  # Total number of picks

while not done and pick_count < max_picks:
    state = np.reshape(state, [1, 6])  # Adjusted to match input dimensions
    probabilities = model.predict(state).flatten()
    probabilities = np.nan_to_num(probabilities, nan=0.0)
    probabilities /= np.sum(probabilities)
    if np.isnan(probabilities).any() or np.any(probabilities < 0):
        action = random.randrange(len(env.available_players))
    else:
        action = np.random.choice(len(env.available_players), p=probabilities[:len(env.available_players)])
    
    state, reward, done, _ = env.step(action)
    env.render()
    pick_count += 1  # Increment pick counter

    if pick_count >= max_picks:
        done = True